In [2]:
%run "0. config.ipynb"

1887


# data preparation

In [6]:
list_types = list(df_raw.type.unique())

## counting number of events per player ~ game

In [7]:
def compute_players_type_events(logs):
  df = logs[["playerId", "type"]].copy()
  df["count"] = 1

  df = df.groupby(["playerId", "type"]).agg({ "count" : np.sum })

  df = df.unstack()
  df.columns = df.columns.droplevel()
  df = df.reset_index()
  df = df.fillna(0)
  df = df.set_index([ "playerId" ])
#   print len(df.index)
#   print df.columns
  
  return df

In [8]:
players_stats = compute_players_type_events(df_raw)

## compute game durations

In [9]:
def compute_game_durations(logs):

  df = logs.groupby("playerId").agg({ "serverTime": [ np.min, np.max  ] })

  df["duration"] = pd.to_datetime(df["serverTime"]["amax"]) - pd.to_datetime(df["serverTime"]["amin"])
  df["duration"] = df["duration"].map(lambda x: np.timedelta64(x, 's'))

  df = df.loc[:, [("duration", "")]]

  df.columns = df.columns.droplevel()
  df.columns = [ "duration (seconds)" ]

  df["duration (seconds)"] = df["duration (seconds)"].astype(int)/1000000000
  
  return df

In [10]:
players_stats = pd.merge(players_stats, compute_game_durations(df_raw), left_index=True, right_index=True)

## compute max checkpoint reached by players

In [11]:
def max_reach(x):
  checkpoints = [ int(checkpoint[-2:]) for checkpoint in x if checkpoint != np.nan and not isinstance(checkpoint, float)]
  
  # print checkpoints
  
  if len(checkpoints) > 0:
    return max([ int(checkpoint[-2:]) for checkpoint in x if checkpoint != np.nan and not isinstance(checkpoint, float)])
  else:
    return 0

def adventure(x):
  return "adventure1" in [ w.split(".")[0] for w in x if w != np.nan and not isinstance(w, float) ]

def sandbox(x):
  return "sandbox1" in [ w.split(".")[0] for w in x if w != np.nan and not isinstance(w, float) ]

def compute_max_reachpoint(logs):
  df = logs.loc[:, ["playerId", "type", "section"]].groupby("playerId").agg({ "section": [ max_reach, adventure, sandbox ]  })
  
  df.columns = df.columns.droplevel()
  
  return df

compute_max_reachpoint(df_raw[0:500]).head()

,max_reach,adventure,sandbox
playerId,,,
0306db66-081d-4035-b30f-8358469d6ec3,8,True,False
09298114-1161-476c-8243-915709d6056e,7,False,False
0be21833-f660-4b7e-9ce5-fc5acf981d8f,3,True,False
10780614-9c63-4cdf-b0c0-4e77c6d2d9c1,0,False,False
15086510-8422-4d25-b345-cf2760b3e48d,0,False,False


In [12]:
players_stats = pd.merge(players_stats, compute_max_reachpoint(df_raw), left_index=True, right_index=True)

In [17]:
len(players_stats[ (players_stats["adventure"] == False) & (players_stats["sandbox"] == False) ])

60

In [16]:
len(players_stats[ (players_stats["adventure"] == True) & (players_stats["sandbox"] == True) ])

0

## preview

In [13]:
players_stats.head()

,complete,configure,craft,death,equip,gotomooc,gotourl,pickup,reach,restart,selectmenu,start,switch,unequip,duration (seconds),max_reach,adventure,sandbox
playerId,,,,,,,,,,,,,,,,,,
0023dbb1-7f98-4cdb-8122-722f801f40b3,0,1,0,3,0,0,0,1,2,0,1,0,0,0,175,2,True,False
01b0c435-f0c0-4bfd-9189-86fc0d29b163,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,False,False
02bc076b-32aa-467a-bbc6-b746abedb7bd,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,False,False
02c6953a-0417-4858-8efb-1989be9f6b9d,0,1,0,0,0,0,0,0,1,0,2,1,1,0,19,1,False,False
0306db66-081d-4035-b30f-8358469d6ec3,1,0,3,17,8,0,1,5,12,1,2,0,0,2,1030,8,True,False


In [14]:
players_stats.to_csv("data/players_stats.csv", encoding="utf-8")